# Data cleaning
### Prerequisites

In [1]:
import pandas as pd
import re
from langdetect import detect, LangDetectException
import emoji

### Load the datasets

In [2]:
df_comments = pd.read_csv("youtube_comments.csv")

### Preprocessing YouTube comments

In [3]:
# Convert to datetime
df_comments['video_date'] = pd.to_datetime(df_comments['video_date'])
df_comments['comment_date'] = pd.to_datetime(df_comments['comment_date'])

# Drop time
df_comments['video_date'] = df_comments['video_date'].dt.date
df_comments['comment_date'] = df_comments['comment_date'].dt.date

In [4]:
# Function for language check
def is_english(text):
    """
    Returns True if the text contains any English tokens.
    Emoji-only or punctuation-only comments are allowed.
    """
    # Extract alphabetic words
    words = re.findall(r'[a-zA-Z]+', text)

    # If there are no words (emoji-only, punctuation-only) → keep it!
    if len(words) == 0:
        return True

    # If there are English words, run detection
    try:
        lang = detect(" ".join(words))
        return lang == 'en'
    except LangDetectException:
        return False

In [5]:
def clean_text(text):
    """
    Clean text for ML while preserving emojis and English comments.
    - Keep emojis
    - Keep punctuation
    - Remove URLs
    - Keep spacing normal
    """
    text = str(text)

    # Remove URLs
    text = re.sub(r"http\S+|www\S+", "", text) 
    # Remove decorative symbols
    text = re.sub(
        r'[^\w\s.,!?\'"@#\$%&\*\-–—…'  # standard punctuation
        r'\U0001F300-\U0001F5FF'       # Misc Symbols and Pictographs
        r'\U0001F600-\U0001F64F'       # Emoticons
        r'\U0001F680-\U0001F6FF'       # Transport & Map Symbols
        r'\U0001F700-\U0001F77F'       # Alchemical Symbols
        r'\U0001F780-\U0001F7FF'       # Geometric Shapes Extended
        r'\U0001F800-\U0001F8FF'       # Supplemental Arrows-C
        r'\U0001F900-\U0001F9FF'       # Supplemental Symbols and Pictographs
        r'\U0001FA00-\U0001FA6F'       # Chess Symbols, etc.
        r'\U0001FA70-\U0001FAFF'       # Symbols & Pictographs Extended-A
        r']', '', text)
    # Normalize whitespace
    text = re.sub(r"\s+", " ", text).strip()   

    if not is_english(text):
        return ""   # skip non-English comments

    return text

df_comments['clean_comment'] = df_comments['comment'].apply(clean_text)

### Descriptive analysis

In [6]:
# Unique videos
print(f"Number of videos: {df_comments['video_id'].nunique()}")

# Total comments
print(f"Total comments: {len(df_comments)}")

# English comments
num_english = (df_comments['clean_comment'] != "").sum()
print("Number of English comments after cleaning:", num_english)

# Date range
print(f"Comment date range: {df_comments['comment_date'].min()} to {df_comments['comment_date'].max()}")

Number of videos: 49
Total comments: 7042
Number of English comments after cleaning: 6594
Comment date range: 2025-10-29 to 2025-12-04


In [7]:
df_comments.columns

Index(['video_id', 'video_title', 'video_url', 'video_description',
       'video_date', 'comment', 'comment_likes', 'comment_date',
       'clean_comment'],
      dtype='object')

In [8]:
# pd.set_option("display.max_colwidth", None) - turn ON full text
# pd.reset_option("display.max_colwidth") - turn OFF full text
df_comments['clean_comment'].head()

0                     I just want everyone to be happy
1    My wife 30 yo bi woman recently told me that s...
2               💙𝗠𝗔𝗞𝗘 𝗠𝗔𝗟𝗘 𝗟𝗢𝗡𝗘𝗟𝗜𝗡𝗘𝗦𝗦💙 🦅 𝗪𝗢𝗥𝗦𝗘 𝗔𝗚𝗔𝗜𝗡 🦅
3    I was a happily married woman for most of my l...
4    Back when I was in high school I have been pra...
Name: clean_comment, dtype: object

### Save to CSV

In [9]:
df_comments.to_csv("youtube_comments_clean.csv", index=False)